# Задание

In [2]:
import numpy as np
import pandas as pd

from pandasql import sqldf

In [162]:
# Сгенерируем 30 поездов и 500 вагонов

Carnumber = np.arange(1, 501)
Car_type = ['цистерна', 'полувагон', 'лесовоз']
Train_index = np.arange(1, 10)
Weight = np.arange(10, 100)
Road = np.arange(1000, 10000)
n = 500 

In [163]:
Car_type = np.random.choice(Car_type, n)
Train_index = np.random.choice(Train_index, n)
Weight = np.random.choice(Weight, n)
Road = np.random.choice(Train_index, n)

In [525]:
# И запишем данные в датафрейм

T1 = pd.DataFrame({'Carnumber': Carnumber,
                   'Car_type': Car_type,
                   'Train_index': Train_index,
                   'Weight': Weight,
                   'Road': Road})

In [526]:
T1

,Carnumber,Car_type,Train_index,Weight,Road
0,1,полувагон,3,59,6
1,2,полувагон,2,49,7
2,3,лесовоз,5,86,7
3,4,полувагон,1,10,6
4,5,цистерна,5,65,6
...,...,...,...,...,...
495,496,лесовоз,2,48,6
496,497,полувагон,4,50,2
497,498,лесовоз,4,81,2
498,499,полувагон,1,23,7


<img src="http://d1vt1c82ljabfd.cloudfront.net/images/sql-i-love.jpg" width="200" height="200" />

In [989]:
Max_weights_p = f"""SELECT Carnumber, Weight
FROM T1
WHERE
    Weight =
    (SELECT MAX(Weight)
    FROM T1)"""

In [990]:
sqldf(Max_weights_p)

,Carnumber,Weight
0,186,99
1,224,99
2,229,99
3,258,99
4,328,99
5,418,99
6,429,99
7,438,99
8,462,99


In [985]:
More_than_60_k = f"""SELECT Train_index
FROM T1
GROUP BY Train_index
HAVING COUNT(Carnumber) > 60"""

In [986]:
sqldf(More_than_60_k)

,Train_index
0,2
1,4


In [987]:
Road_percentage_13 = f"""
SELECT Road, Car_type,
    100*num_types/sum(num_types) OVER(PARTITION BY Road) AS share
    FROM
    (
    SELECT Road, Car_type,
    count(Carnumber) AS num_types
    FROM T1
    GROUP BY Road, Car_type
    )
    """

In [988]:
sqldf(Road_percentage_13)

,Road,Car_type,share
0,1,лесовоз,33
1,1,полувагон,31
2,1,цистерна,34
3,2,лесовоз,33
4,2,полувагон,29
5,2,цистерна,37
6,3,лесовоз,34
7,3,полувагон,25
8,3,цистерна,40
9,4,лесовоз,27


# Задание 2

In [799]:
#T2
from datetime import datetime, timedelta

Carnumber2 = np.arange(1, 20)
Code_station = ['MCS_KZ', 'MCS_OK', 'RZ_KAZ',
               'SAM_KZ', 'SPB_MS', 'EKB_U', 'KRAS_Y']
Date = np.arange(datetime(2020,3,1), datetime(2020,3,13), timedelta(days=1)).astype(datetime)
n = 300 

Carnumber2 = np.random.choice(Carnumber2, n)
Code_station = np.random.choice(Code_station, n)
Date = np.random.choice(Date, n)

T2 = pd.DataFrame({'Carnumber': Carnumber2,
                   'Code_station': Code_station,
                   'Date': Date})

#T3
data = {'Code_station': ['MCS_KZ', 'RZ_KAZ', 'SAM_KZ',
                                  'MCS_OK', 'SPB_MS',
                                  'EKB_U', 'KRAS_Y'],
        'Code_railway': ['83', '83', '83', '84', '84', '27', '27']}
T3 = pd.DataFrame.from_dict(data)

In [800]:
T2

,Carnumber,Code_station,Date
0,19,KRAS_Y,2020-03-02
1,9,SAM_KZ,2020-03-05
2,19,EKB_U,2020-03-05
3,11,KRAS_Y,2020-03-05
4,11,RZ_KAZ,2020-03-10
...,...,...,...
295,4,MCS_OK,2020-03-09
296,11,RZ_KAZ,2020-03-01
297,16,MCS_KZ,2020-03-02
298,10,SAM_KZ,2020-03-02


In [801]:
T3

,Code_station,Code_railway
0,MCS_KZ,83
1,RZ_KAZ,83
2,SAM_KZ,83
3,MCS_OK,84
4,SPB_MS,84
5,EKB_U,27
6,KRAS_Y,27


In [756]:
# Поработать со временем (8 марта независимо от часов)

March_8th = f"""
SELECT Carnumber, Date, Code_railway, Time
FROM
    (SELECT Carnumber, Date, T3.Code_railway,
    DATEDIFF(day, LAG(Date, 1) OVER(ORDER BY Carnumber), Date) AS Time_diff
    FROM T2
    INNER JOIN T3
    ON T2.Code_station = T3.Code_station
    GROUP BY Carnumber
    )
WHERE Date = '2020-03-08 00:00:00.000000'
    AND Code_railway = '83'
"""

In [1014]:
carriages_8th_march = f"""
SELECT
    Carnumber, Code_railway, Date,
    Code_railway - LAG(Code_railway,1) OVER (PARTITION BY Carnumber
             ORDER BY Date DESC
             RANGE BETWEEN CURRENT ROW AND 1 Following) AS diff
FROM T2
INNER JOIN T3
    ON T2.Code_station = T3.Code_station
WHERE
    Date >= '2020-03-08 00:00:00.000000'
WINDOW W AS (PARTITION BY Carnumber
             ORDER BY Date)
"""

In [1062]:
carriages_8th_march_road_83 = f"""
SELECT
    Carnumber,
    Code_railway,
    ABS(time_on_83) AS time_on_83
FROM
    (
    SELECT
        Carnumber, Code_railway, Date,
        Date - LAG(Date,1) OVER (PARTITION BY Carnumber
             ORDER BY Date DESC
             RANGE BETWEEN CURRENT ROW AND 1 Following) AS time_on_83
    FROM T2
    INNER JOIN T3
        ON T2.Code_station = T3.Code_station
    WHERE
        Date >= '2020-03-08 00:00:00.000000'
    )
WHERE Code_railway = '83'
"""

In [1063]:
sqldf(carriages_8th_march_road_83)

,Carnumber,Code_railway,time_on_83
0,1,83,0.0
1,2,83,0.0
2,2,83,0.0
3,2,83,0.0
4,2,83,0.0
5,2,83,0.0
6,3,83,NaN
7,3,83,0.0
8,4,83,NaN
9,5,83,NaN
